ORF Finding Method

The ORF finder is primarily composed of three functions

*   codons(seq)
*   ORF(condonPositions)
*   ORFfinder(dta, nucleotides, sel_gen, rev)

codons() will find all positions of start and stop codons and place them into three lists, one for each frame. Once found, the ORF() function will accept these lists and validate/pair the start and stop codons based on a set minimum length as well as where the previous ORF was found. For each valid start codon, multiple valid stop codons are paired in order to obtain 3 ORFs. Both of these functions are called repeatedly within the ORFfinder() function. Within this function, the ORFcheck() function also runs to determine how many genes the ORF finder finds and how many extraneous ORFs it finds.

In [ ]:
## Upload data from Module 6 data folder: 

# all_gene_positions.txt
# all_incorrect_ORFS.csv
# full_sequence.fasta
# genome1.fasta
# genome1_gene_positions.txt
# genome2.fasta
# rev_comp.fasta

<font color='blue'>Cell 1 Installing and loading necessary packages

In [ ]:
#install.packages("dplyr")
library("dplyr") #The dplyr package is commonly employed for dataframe manipulation

<font color='blue'>Cell 2 
Reading data

In [ ]:
#Import gene position data 
#The first element of each row is the start position of the gene
#The second is the end position of the gene
all_gene_data <- read.table("all_gene_positions.txt", header=FALSE, sep=",", quote="") 
gene_position <- as.data.frame(all_gene_data[1:2269,]) #Data for forward direction
gene_position_r <- as.data.frame(all_gene_data[2270:4584,]) #Data for reverse complement

#Read full forward direction genome
full_seq <- read.table("full_sequence.fasta", header=FALSE, sep="\n", quote="", stringsAsFactors=FALSE) 
full_seq <- as.data.frame(full_seq[-1,], stringsAsFactors=FALSE) #Remove fasta descriptor (first row)
full_seq <- paste(full_seq[,1], sep="\n", collapse="") #Turn data into one character string by removing line returns

#Read full reverse complement direction genome
full_rev <- read.table("rev_comp.fasta", header=FALSE, sep="\n", quote="", stringsAsFactors=FALSE) 
full_rev <- as.data.frame(full_rev[-1,], stringsAsFactors=FALSE) #Remove fasta descriptor (first row)
full_rev <- paste(full_rev[,1], sep="\n", collapse="") #Turn data into one character string by removing line returns



# ORF Finder

<font color='blue'>Cell 3
Finding codon positions

codons(): This function will find the positions at which a start codon or stop codon appears. This function will return two lists (starts and stops), each with three sublists that contain the found positions in three frames.

Inputs: 
-   seq: The sequence being considered, ie. forward or reverse complement sequences (full_seq and full_rev, respectively)
-   start_c: The start codon considered. This function is called from another function that iterates through ATG, GTG, TTG

Inside the function:
-   stops: The stop codons that will be found
-   lst1: A list that will contain the start codon positions
-   lst2: A list that will contain the stop codon positions
      - Both lst1 and lst2 are a list of 3 lists where each sublist is the list of positions for each frame.

Output:
Returns the list codonPositions which contains lst1 and lst2 

In [ ]:
#ORF Finder part 1: Find codon positions
codons <- function(seq, start_c) {
  
  stops <- c("TAA","TGA","TAG")
  codons <- list() #Empty list that will hold nucleotide triplets (codons)
  lst1 <- list() #Empty list that will hold 3 lists of start positions, one list for each reading frame
  lst2 <- list() #Empty list that will hold 3 lists of stop positions, one list for each reading frame
  
  #Split sequence into reading frames using the sub() function, which removes unread nucleotides at the start of the sequence
  frame1 <- seq #First frame starts reading at first nucleotide
  frame2 <- sub(".", "", seq) #Second frame starts reading at second nucleotide
  frame3 <- sub("..", "", seq) #Third frame strats reading at thrid nucleotide
  frames <- list(frame1, frame2, frame3) #Create list of 3 reading frames
  
  #Cut frames into triplets and store in the list codons
  #The functions gregexpr() and regmatches() find and extract the triplets, respectively
  for (i in 1:length(frames)){ 
    codons[[i]] <- regmatches(frames[[i]], gregexpr(".{3}", frames[[i]]))[[1]]
  }
  
  #Postion of all start codons from all frames
  #Start by producing lists of the positions of each start codon in each frame using the grep() function
  for (i in 1:length(start_c)) { 
    lst1[[i]] <- list(3*grep(pattern=start_c[i], codons[[1]])-2, 3*grep(pattern=start_c[i], codons[[2]])-1, 3*grep(pattern=start_c[i], codons[[3]])) #Multiply the codon position by 3 and adjust for reading frame to obtain the nucleotide position of each start codon
  }
  #Then combine start codon lists to obtain one list of all start codons for each frame using the mapply() function
  lst1 <- mapply(c, lst1[[1]], lst1[[2]], lst1[[3]]) 
  
  #Repeat above for stop codons
  for (i in 1:length(stops)) {
    lst2[[i]] <- list(3*grep(pattern=stops[i], codons[[1]])-2, 3*grep(pattern=stops[i], codons[[2]])-1, 3*grep(pattern=stops[i], codons[[3]]))
  }
  lst2 <- mapply(c, lst2[[1]], lst2[[2]], lst2[[3]])
  
  #Finally return lst1 and lst2 in a list called codonPositions. We must do this because R functions can only return one element.
  codonPositions <- list(lst1, lst2)
  return(codonPositions)
  
}

<font color='blue'>Cell 4
Determining valid ORFs

ORF(): Pairs the start and stop codon positions found with the codons function to form valid ORF start-stop pairs.

Inputs:
*   codonPositions is the output of the codons() function
*   min_len is the minimum length used for determining valid ORFs

Inside the function:
*   ORFstarts and ORFstops: Lists that will contain the starts and stops of the determined valid ORFs
*   all_frames: When looking for ORFs, all start codon positions for all frames will be put into one list. This allows to scan for valid ORF starts sequentially across all frames.
*   min_len: The minimum ORF length is 40
*   start_min: Starting from zero, this variable is updated to dictate the position from which the next ORF start will be considered. After a start codon has been paired with its stop codons, this variable is updated with the min_len and the look_back.
*   stop_count: The stop position for loop iterates until each valid start codon is paired with 3 stops so long as the min_len condition is met. 
*   look_back: This parameter is subtracted from the start_min variable to account for the possibility of overlapping ORFs

Output: 
-   ORFs: a list that contains ORFstarts, ORFstops and found_frame. ORFstarts and ORFstops contain the start and stop positions of codons that form true ORFs. Each element in the ORFstarts list is paired with the corresponding element of the ORFstops list, eg. ORFstarts[1] and ORFstops[1] are the indices of the first ORF that was found by the ORF() function. The third vector, found_frame, contains the reading frames for all valid start codons (only used for diagnostic purposes).


In [ ]:
#ORF Finder part 2: Determine valid ORFs
ORF <- function(codonPositions, min_len) {
  
  ORFstarts <- as.numeric() #Empty vector for start positions
  ORFstops <- as.numeric() #Empty vector for stop positions
  found_frame <- as.numeric() #Empty vector for reading frames of all valid start codons
  look_back <- 5 #To account for overlapping ORFs
  start_min <- 0 #Set minimum start position
  #min_len <- 40
  
  #Consolidate and sort start and stop codon positions in ascending order
  lst1 <- lapply(codonPositions[[1]], sort) #Maintain 3 lists for reading frames, each list in ascending order
  lst2 <- lapply(codonPositions[[2]], sort) #Maintain 3 lists for reading frames, each list in ascending order
  all_frames <- sort(unlist(lst1)) #Consolidate start codon positions into one vector, in ascending order

  for (s_cod in all_frames) { 
        
    for (s_frame in 1:length(lst1)) {  #Find the frame of the start codon
      if (s_cod %in% lst1[[s_frame]]) {
        frame <- s_frame
        found_frame <- append(found_frame, frame) #Add the frame to the vector found_frame
        }
      }
      
    stop_count <- 1 #Set number of stop positions to begin at 1 for each start codon tested 

    for (stp_cod in lst2[[frame]]) {
      if (s_cod > start_min) { #Start looking for stops in the same frame as the start codon
        if (stop_count > 3) { #The minimum start position will be reset after finding 3 stop codons
          start_min <- start_min + min_len - look_back #Start position will be less than the new minimum start position, so the stop position "for" loop will break and the next start codon will be tested
        }
          
        else {
          if (stp_cod > s_cod + min_len) { #Before breaking out of the stop position "for" loop, add stop position to accompany the start position in corresponding lists ORFstops and ORFstarts 
            ORFstarts <- append(ORFstarts, s_cod)
            ORFstops <- append(ORFstops, stp_cod)
            stop_count <- stop_count + 1 #Move to next corresponding stop codon
          }
        }
      }
    }
  }

  ORFs <- list(ORFstarts, ORFstops, found_frame)
  return(ORFs)

}

<font color='blue'>Cell 5
ORF checking function

ORFcheck(): Checks which of the ORF start-stop pairs are coding (ie. found in the gene position data) and which are non coding (ie. not found in the gene position data).

Inputs:
*   ORFs from the ORF() function
*   gene_pos: The gene position data (either gene_position or gene_position_r)
*   rev: A TRUE/FALSE parameter that is set to TRUE if looking at the reverse complement (with gene_position_r), and FALSE otherwise

Inside the function:
-   OC: start-stop pair being tested
*   corr: The number of found ORFs that are contained in the gene position data (used for diagnostic purposes)
*   corORF: A dataframe of the ORFs (start and end positions) that are contained in the gene position data
*   incorORF: A dataframe of the ORFs (start and end positions) that are not contained in the gene position data (non coding ORFs)

Output: Returns ORFcheckData, a list containing corr, corORF and incorORF.

In [ ]:
#ORF Finder part 3: Determine which found ORFs are coding and non coding
ORFcheck <- function(ORFs, gene_pos, rev) {
  
  ORFstarts <- ORFs[[1]] 
  ORFstops <- ORFs[[2]] 

  if (rev == TRUE) {
    ORFfound <- data.frame(ORFstarts-1, ORFstops+2) #Different indexing than forward genome. Add 2 to stop position to obtain position of last nucleotide in stop codon.
  }
  
  else {
    ORFfound <- data.frame(ORFstarts, ORFstops+2) #Add 2 to stop position to obtain position of last nucleotide in stop codon.
  }  
  
  colnames(ORFfound) <- c("ORFstarts", "ORFstops") #Renaming the columns will enable us to use the dplyr join functionalities to find the coding and non coding ORFs.
  colnames(gene_pos) <- c("ORFstarts", "ORFstops")
  corORF <- inner_join(ORFfound, gene_pos, by=c("ORFstarts", "ORFstops")) #The inner join yields all ORFs that were found and coding (ie. found in the gene position data)
  incorORF <- anti_join(ORFfound, gene_pos, by=c("ORFstarts", "ORFstops")) #The anti join yields all ORFs that were found and non coding (ie. not found in the gene position data)
  corr <- nrow(corORF) #Count the number of correct ORFs
  ORFcheckData <- list(corr, corORF, incorORF)
  return(ORFcheckData)
  
}

<font color='blue'>Cell 6
Running the previous functions

ORFfinder(): This function simply runs the functions defined above and returns the dataframes of all coding and non coding ORFs found

Inputs:
*   dta: Either full_seq or full_rev (forward genome sequence or reverse complement genome sequence)
*   nucleotides: A parameter to limit the ORFfinder search to reduce processing time (ie. nucleotides = 10000 will only look at the first 10000 nucleotides in the genome)
*   sel_gen: The gene position data that is also limited by the 'nucleotides' parameter
*  rev and min_len as defined in previous functions

Outputs: Returns the list ORFfinderData which is a lists composed of two dataframes, one that holds the start-stop pairs for coding ORFs (corORF) and one that holds the start-stop pairs for non coding ORFs (incorORF)

In [ ]:
#ORF Finder part 4: Putting all functions together
ORFfinder <- function(dta, nucleotides, sel_gen, rev, min_len) {

  start_codons <- c("ATG","GTG","TTG")
  codonPositions <- codons(dta[1:nucleotides], start_codons)
  ORFs <- ORF(codonPositions, min_len)
  ORFcheckData <- ORFcheck(ORFs, sel_gen, rev)
  ORFfinderData <- list(ORFcheckData[[2]], ORFcheckData[[3]]) #Note ORFcheckData[[2]] is the dataframe corORF (coding ORFs), while ORFcheckData[[3]] is the dataframe incorORF (non coding ORFs)
  return(ORFfinderData)
  
}

# ORF Finder with an E. coli genome



<font color='blue'>Cell 7
Finding ORFs on a section of the E. Coli genome

genome_section(): This function takes inputs from the user and runs the ORFfinder() function. This function prints how many coding ORFs and non coding ORFs were found.

Output: Returns a list called all_pred which contains 2 dataframes, one that contains all start-stop pairs for coding and non coding ORFs one the forward genome (all_f_pred), and one that contains all start-stop pairs for coding and non coding ORFs one the reverse complement genome (all_r_pred).

In [ ]:
genome_section <- function() {
  #Get the necessary parameters from the user
  nucleotides <- readline(prompt="Please enter the number of nucleotides to analyze: ")
  nucleotides <- as.numeric(nucleotides) #Convert from a string to a numeric
  min_len <- readline(prompt="Please enter the minimum length of ORFs to consider: ")
  min_len <- as.numeric(min_len)
  
  #Limit the forward and reverse complement using the 'nucleotides' parameter. The substr() function subsets the genome to obtain nucleotides with indices 1 to the nucleotides threshold.
  forward_section <- substr(full_seq, start=1, stop=nucleotides)
  reverse_section <- substr(full_rev, start=1, stop=nucleotides)
  
  #Limiting the gene position data by the 'nucleotides' parameter 
  #The apply() function assigns TRUE to rows (row is indicated by the 1 argument of the apply() function) when any value in the row is greater than the 'nucleotides' parameter, and FALSE otherwise. By using the logical NOT operator (!) before the apply() function, we are limiting the gene position data to ORFs that are found within our nucleotide threshold.
  selected_genes_f <- gene_position[!apply(gene_position, 1, function(x) any(x > nucleotides)),] 
  selected_genes_r <- gene_position_r[!apply(gene_position_r, 1, function(x) any(x > nucleotides)),]

  #Running the ORFfinder() function on the forward and reverse complement with the user entered parameters
  f_ORFfinderData <- ORFfinder(dta=forward_section, nucleotides=nucleotides, sel_gen=selected_genes_f, rev=FALSE, min_len=min_len)
  r_ORFfinderData <- ORFfinder(dta=reverse_section, nucleotides=nucleotides, sel_gen=selected_genes_r, rev=TRUE, min_len=min_len)
  
  #Consolidate results
  #Note f_ORFfinderData[[1]] is a dataframe of coding ORFs, while f_ORFfinderData[[2]] is a dataframe for non coding ORFs
  all_f_pred <- list(f_ORFfinderData[[1]], f_ORFfinderData[[2]])
  all_r_pred <- list(r_ORFfinderData[[1]], r_ORFfinderData[[2]])
  all_pred <- list(all_f_pred, all_r_pred)

  #return(all_pred)

  #Printing the number of genes found within the nucleotides threshold, the number of coding ORFs and non coding ORFs found using the ORFfinder() function
  cat(paste("-----------------------------------------------------------", 
    paste0("There are ", nrow(selected_genes_f), " genes on the forward strand"), 
    paste0("There are ", nrow(selected_genes_r), " genes on the reverse strand"),
    "-----------------------------------------------------------",
    "Forward:", 
    paste0(length(f_ORFfinderData[[1]][[1]]), " found correctly"), 
    paste0(length(f_ORFfinderData[[2]][[1]]), " additional ORFs found"),
    "-----------------------------------------------------------",
    "Reverse:", 
    paste0(length(r_ORFfinderData[[1]][[1]]), " found correctly"), 
    paste0(length(r_ORFfinderData[[2]][[1]]), " additional ORFs found"),
    sep="\n")) #This is one technique for printing text as the output of a function

}

genome_section()

<font color='blue'>Cell 8
Running the previous functions on a fraction of the E. coli genome

This code block outputs the number of ORFs found along with tables of the first 5 ORFs for both the forward and the reverse complement genome sequences. It also contains two functions, test_genome_ORFs (), which obtains a fraction of the imported genome sequence for ORF testing, and rev_comp(), which obtains the reverse complement sequence of the imported genome sequence.

In [ ]:
genome_filename <- "genome1.fasta"

#The test_genome_ORFs() function is used to obtain a fraction of the original genome for ORF detection. Use this to reduce processing time.
test_genome_ORFs <- function(lim) { #The limit argument will be a float between 0 and 1
  test_genome <- as.character(read.table(file=genome_filename, header=FALSE, sep="\n", quote="", stringsAsFactors=FALSE))
  test_genome <- substr(test_genome, start=1, stop=nchar(test_genome)*lim)
  return(test_genome)
}

#Finding reverse complement of genome sequence with the rev_comp() function 
rev_comp <- function(test_genome) {
  test_genome_split <- strsplit(test_genome, "")[[1]] #Split genome into single characters
  test_genome_reversed <- rev(test_genome_split) #Reverse the split genome
  test_gen_rcomp <- character() #Create an empty character vector that will become the reverse complement sequence

  for (i in test_genome_reversed) {
  
    if (i == "A") {
      j <- "T"
    }
  
    else if (i == "T") {
      j <- "A"
    }
  
    else if (i == "C") {
      j <- "G"
    }
  
    else {
      j <- "C"
    }
  
    test_gen_rcomp <- append(test_gen_rcomp, j)
  
  }

  test_genome_r <- paste(test_gen_rcomp, collapse="") #Consolidate the single characters into one string
  return(test_genome_r)

}

size_limit <- readline(prompt="Enter a value between 0 and 1 to limit the genome: ")
size_limit <- as.numeric(size_limit)
min_len <- readline(prompt="Please enter the minimum length of ORFs to consider: ")
min_len <- as.numeric(min_len)

test_genome <- test_genome_ORFs(size_limit)

#Find ORFs for forward genome
start_codons <- c("ATG","GTG","TTG")
codonPositions <- codons(test_genome, start_codons)
ORFs <- ORF(codonPositions, min_len)
ORFstarts <- ORFs[[1]]
ORFstops <- ORFs[[2]]
forward_ORFs <- data.frame(ORFstarts, ORFstops) #Create dataframe to hold the start-stop pairs
colnames(forward_ORFs) <- c("ORF start location", "ORF stop location") #Rename the columns using the colnames() function

#Repeat for reverse compliment genome
test_genome_r <- rev_comp(test_genome)
codonPositions_r <- codons(test_genome_r, start_codons)
ORFs_r <- ORF(codonPositions_r, min_len)
ORFstarts_r <- ORFs_r[[1]]
ORFstops_r <- ORFs_r[[2]]
reverse_ORFs <- data.frame(ORFstarts_r, ORFstops_r)
colnames(reverse_ORFs) <- c("ORF start location", "ORF stop location")

#This is another technique for printing text as the output of a function 
noquote("-----------------------------------------------------------")
noquote(paste0("ORFs found on the forward direction:            ", nrow(forward_ORFs)))
noquote(paste0("ORFs found on the reverse complement direction: ", nrow(reverse_ORFs)))

noquote("-----------------------------------------------------------")
noquote("The first 5 ORFs found on the forward direction: ")
noquote(head(forward_ORFs, n=5))

noquote("-----------------------------------------------------------")
noquote("The first 5 ORFs found on the reverse complement direction: ")
noquote(head(reverse_ORFs, n=5))
